In [ ]:
from svg.path import parse_path, Line, Move, CubicBezier
from xml.dom.minidom import parse, Document
from simplification.cutil import simplify_coords, simplify_coords_vw

# Parse the SVG file
dom = parse('../../lens/src/assets/img/uk-outline.svg')
path_strings = [path.getAttribute('d') for path in dom.getElementsByTagName('path')]

# Create a new SVG document
new_dom = Document()
svg_element = new_dom.createElement('svg')
new_dom.appendChild(svg_element)

for path_string in path_strings:
    path_data = parse_path(path_string)

    # Simplify curves to straight lines
    for i in range(len(path_data)):
        segment = path_data[i]
        if isinstance(segment, CubicBezier):
            # Replace the curve with a straight line
            path_data[i] = Line(segment.start, segment.end)

    # Convert the path to a list of points
    points = [(point.start.real, point.start.imag) for point in path_data if not isinstance(point, Move)]

    # Simplify the path using the Ramer-Douglas-Peucker algorithm
    simplified_points = simplify_coords(points, 5.0)  # Increase epsilon to 5.0

    # Convert the simplified points back to a path
    simplified_path_data = ' '.join(f'L {point[0]} {point[1]}' for point in simplified_points)
    simplified_path_data = 'M' + simplified_path_data[1:]  # Replace the first 'L' with 'M'

    # Create a new path element with the simplified path
    path_element = new_dom.createElement('path')
    path_element.setAttribute('d', simplified_path_data)
    svg_element.appendChild(path_element)

# Write the new SVG document to a file
with open('uk_outline_01.svg', 'w') as file:
    file.write(new_dom.toprettyxml())